In [4]:
!pip install ace_tools

In [277]:
# Re-import after code state reset
from faker import Faker
import pandas as pd
import sqlite3
import uuid
import random
from datetime import datetime

# Initialize Faker
fake = Faker()

# Generate synthetic customer data
def generate_customers(n=70):
    data = []
    for _ in range(n):
        customer = {
            "id": str(uuid.uuid4()),
            "full_name": fake.name(),
            "email": fake.email(),
            "phone_number": fake.phone_number(),
            "address": fake.address().replace("\n", ", "),
            "account_number": fake.unique.bban(),
            "account_type": random.choice(["Savings", "Cheque", "Business"]),
            "balance": round(random.uniform(100.0, 50000.0), 2),
            "loan_eligible": random.choice([True, False]),
            "loan_amount": round(random.uniform(0.0, 200000.0), 2),
            "last_login": fake.date_time_between(start_date="-1y", end_date="now").strftime("%Y-%m-%d %H:%M:%S"),
        }
        data.append(customer)
    return pd.DataFrame(data)

# Generate the data
df_customers = generate_customers()

# Save to SQLite database in the Jupyter environment
conn = sqlite3.connect("local_customers.db")
df_customers.to_sql("customers", conn, index=False, if_exists="replace")
conn.commit()
conn.close()

# import ace_tools as tools; tools.display_dataframe_to_user(name="Local Synthetic Customer Data", dataframe=df_customers)


In [278]:
df_customers.head()

,id,full_name,email,phone_number,address,account_number,account_type,balance,loan_eligible,loan_amount,last_login
0,000bf41d-edad-461d-bcbc-1b083255e996,Michael Bell,cassidy38@example.com,380-632-3208,"6424 Katherine Plaza, Lisafort, TX 88492",WVTZ78468039103514,Cheque,14137.05,False,87573.25,2024-09-04 01:09:16
1,1cf22e94-ef93-4cb6-9dc7-98fae1ca1385,Rachel Shields,normancunningham@example.com,384.366.1254x58342,"6631 Zavala Forges Suite 346, Mclaughlinbury, ...",UVIN18245880129599,Cheque,26916.94,False,180095.17,2024-06-06 08:31:40
2,1d8e2aab-bb19-4c31-aceb-2a499b43460e,Tara Griffin,santospaul@example.com,+1-344-756-6556x60846,"29134 Lawrence Ferry Suite 904, Loweryville, S...",OUFV22504658577747,Savings,8358.30,True,181958.95,2024-06-29 16:27:09
3,8491bdbd-efa9-4609-b126-98c6a489a221,Keith French,contrerasmaxwell@example.org,+1-354-920-5110x3755,"94475 Jennifer Port, Port Matthewport, DC 86648",IVYR85652857633441,Business,21866.51,True,12203.69,2024-11-11 01:16:21
4,0aecaab0-f5ea-44fe-b469-bef30c84bee2,Laura Jones,duffynicole@example.com,001-772-826-1239x8143,"3245 Donald Land Apt. 878, Sierraton, KS 28325",LOPH47571453162501,Cheque,16331.49,True,178752.80,2025-04-29 09:02:26


# Functions to be called

In [279]:
# Step 1: Set up database connection for reuse
import sqlite3

# Connect to the local SQLite DB
db_path = "local_customers.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# # Function to check if a customer exists by ID or account number
# def get_customer_by_id_or_account(identifier):
#     query = """
#     SELECT * FROM customers WHERE id = ? OR account_number = ?
#     """
#     cursor.execute(query, (identifier, identifier))
    
#     result = cursor.fetchone()
#     if result:
#         columns = [column[0] for column in cursor.description]
#         return dict(zip(columns, result))
#     return None


# Function to update customer details
def update_customer_detail(identifier, field, new_value):
    query = f"""
    UPDATE customers SET {field} = ? WHERE id = ? OR account_number = ?
    """
    cursor.execute(query, (new_value, identifier, identifier))
    conn.commit()
    return cursor.rowcount > 0

# Confirm that everything is ready
"Database connection and utility functions are set up. Ready for GPT chatbot integration."


'Database connection and utility functions are set up. Ready for GPT chatbot integration.'

# First Agent

In [5]:
import openai
import os
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 
openai.api_key = os.getenv("OPENAI_API_KEY")

In [6]:
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [13]:
# Define the system prompt for the update agent
update_prompt = """
You are a helpful and friendly banking assistant chatbot.
Your job is to help users update their personal information: full name, phone number, or address.

Always begin by asking the user to provide their ID or account number.

Once you have the ID:
- Call a function to check if the user exists.
- If the user is found, ask what they want to update.
- Then ask for the new value.
- Call the update function and confirm the update.

Keep the tone professional, polite, and clear.
"""

# Sample conversation simulation using OpenAI ChatCompletion
def ask_gpt_4o(user_message, chat_history=[]):
    
    messages = [{"role": "system", "content": update_prompt}] + chat_history
    messages.append({"role": "user", "content": user_message})

    response = client.responses.create(
        model="gpt-4o-mini",  
        instructions=update_prompt,
        input = user_message,
        temperature=0.0
    )
    
    return response.output_text, messages

"Chat setup complete. Ready to simulate a conversation."


'Chat setup complete. Ready to simulate a conversation.'

In [14]:
first_reply, history = ask_gpt_4o("Hi, I want to update my phone number.")
first_reply

'Sure! To get started, could you please provide your ID or account number?'

In [15]:
history

[{'role': 'system',
  'content': '\nYou are a helpful and friendly banking assistant chatbot.\nYour job is to help users update their personal information: full name, phone number, or address.\n\nAlways begin by asking the user to provide their ID or account number.\n\nOnce you have the ID:\n- Call a function to check if the user exists.\n- If the user is found, ask what they want to update.\n- Then ask for the new value.\n- Call the update function and confirm the update.\n\nKeep the tone professional, polite, and clear.\n'},
 {'role': 'user', 'content': 'Hi, I want to update my phone number.'}]

# Get balance agent

In [139]:
import os
import json
import sqlite3
from openai import OpenAI

from dotenv import load_dotenv

# ─── Load API Key ──────────────────────────────────────────────────────────────
_ = load_dotenv() 
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ─── Your balance‐checking function ───────────────────────────────────────────
def get_balance_from_db(customer_id: str) -> dict:
    """
    Query the local_customers.db for this customer_id 
    and return a dict with name & balance (or an error).
    """
    conn = sqlite3.connect("local_customers.db")
    cur = conn.cursor()
    cur.execute(
        "SELECT full_name, balance FROM customers WHERE id = ?", 
        (customer_id,)
    )
    row = cur.fetchone()
    conn.close()

    if row:
        return {"found": True, "full_name": row[0], "balance": row[1]}
    else:
        return {"found": False}
            

# ─── Tell GPT about your function ─────────────────────────────────────────────
function_descriptions = [
    {
        "type": "function",
        "name": "get_balance",
        "description": "Retrieve a customer's name and balance from the SQLite database",
        "parameters": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "string",
                    "description": "The ID of the customer"
                },
#                 "full_names": {
#                     "type": "string",
#                     "description": "Name of of the customer"
#                 }
                
            },
            "required": ["customer_id"],
        },
    }
]


# ─── The system prompt you already wrote ──────────────────────────────────────
balance_prompt = """
You are a helpful banking assistant. A user wants to check their account balance.

Start by asking them for their account number or ID.

Once you have the ID:
- Check if the customer exists.
- If yes, respond with their name and current balance.
- If not, ask them to double-check.

Only answer bank-related questions

Keep your tone friendly and professional.
"""


In [286]:
get_balance_from_db('000bf41d-edad-461d-bcbc-1b083255e996')

{'found': True, 'full_name': 'Michael Bell', 'balance': 14137.05}

In [ ]:
    input=[{"role": "user", "content": "Can you find information about ChatGPT in the AI knowledge base?"}],


In [297]:
def chat_with_openai(user_input):
    
    response = client.responses.create(
            model="gpt-4o-mini",
            input = [{"role": "user", "content": user_input}],
            
            instructions = balance_prompt,
            tools=function_descriptions,
            tool_choice = "auto",
            temperature = 0.0
        )

    # Return the chatbot's reply
    return response ,response.output_text

In [274]:
chat_with_openai('hi what is my account balance')

(Response(id='resp_6829f82f455081918d745ea6995f10dc0b4202e786e48312', created_at=1747580975.0, error=None, incomplete_details=None, instructions='\nYou are a helpful banking assistant. A user wants to check their account balance.\n\nStart by asking them for their account number or ID.\n\nOnce you have the ID:\n- Check if the customer exists.\n- If yes, respond with their name and current balance.\n- If not, ask them to double-check.\n\nOnly answer bank-related questions\n\nKeep your tone friendly and professional.\n', metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[ResponseOutputMessage(id='msg_6829f82fd9708191bdd0ed240979a5d00b4202e786e48312', content=[ResponseOutputText(annotations=[], text='Hello! I can help you with that. Could you please provide me with your account number or ID?', type='output_text')], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=0.0, tool_choice='auto', tools=[FunctionTool(name='get_balance

In [321]:
def chat_balance():
    
    print(" Welcome! I'm your chatbot. Type 'exit' to end the chat.\n")

    messages = [
        {"role": "system", "content": balance_prompt}
    ]

    response = None
    
    while True:
        
        user_input = input("You: ")
        
        if "exit" in user_input.lower() or "thank you" in user_input.lower() or "bye" in user_input.lower():
            print("Goodbye and have a nice day!")
            break

        
        response, bot_response = chat_with_openai(user_input)
        print(f"Bot: {bot_response}\n")
        
        messages.append({"role": "user", "content": user_input})


        print(f"clean response: {response}")
                
                
        if response:
                
                if response.tools[0].type != "function":
                    continue

                name = response.tools[0].name
                print(f"tool name: {name}")

#                 print(f"args: {response.output[0].arguments}")
                
                print(f"type of output: type({type(response.output[0])}")
                
                func_call = response.output[0]
                output_str = str(func_call)
                print(f"output_str: {output_str}")
                
                customer_id = output_str[52:88]
                print(f"customer_id: {customer_id}")
                

                result = get_balance_from_db(customer_id)
                
                print(f"sql results: {result}")

#     #             result = get_balance_from_db(args["properties"]["customer_id"])

#                 follow_up = client.responses.create(
#                     model="gpt-4o-mini",
#                     input=messages,
#                 )

# #                 final_message = follow_up.choices[0].message.content
#                 final_message = follow_up.output_text
        
#                 print(f"Bot: {final_message}\n")

#                 messages.append({"role": "assistant", "content": final_message})

#                 messages.append(
#                         {
#                             "role": "function",
#                             "name": name,
#                             "output": json.dumps(result),
#                         }
#                     )
        
   




In [322]:
len('000bf41d-edad-461d-bcbc-1b083255e996')

36

In [324]:
chat_balance()

 Welcome! I'm your chatbot. Type 'exit' to end the chat.

You: what is my account balance
Bot: Could you please provide your account number or ID?

clean response: Response(id='resp_682a36ac7f1c81918127ec8b427d5c67069e49daeb10cd98', created_at=1747596972.0, error=None, incomplete_details=None, instructions='\nYou are a helpful banking assistant. A user wants to check their account balance.\n\nStart by asking them for their account number or ID.\n\nOnce you have the ID:\n- Check if the customer exists.\n- If yes, respond with their name and current balance.\n- If not, ask them to double-check.\n\nOnly answer bank-related questions\n\nKeep your tone friendly and professional.\n', metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[ResponseOutputMessage(id='msg_682a36acdef48191a3ed00c1f3827e19069e49daeb10cd98', content=[ResponseOutputText(annotations=[], text='Could you please provide your account number or ID?', type='output_text')], role='assistant', status='complete

KeyboardInterrupt: Interrupted by user

In [294]:
from openai import OpenAI

client = OpenAI()

tools = [{
    "type": "function",
    "name": "search_knowledge_base",
    "description": "Query a knowledge base to retrieve relevant info on a topic.",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "The user question or search query."
            },
            "options": {
                "type": "object",
                "properties": {
                    "num_results": {
                        "type": "number",
                        "description": "Number of top results to return."
                    },
                    "domain_filter": {
                        "type": [
                            "string",
                            "null"
                        ],
                        "description": "Optional domain to narrow the search (e.g. 'finance', 'medical'). Pass null if not needed."
                    },
                    "sort_by": {
                        "type": [
                            "string",
                            "null"
                        ],
                        "enum": [
                            "relevance",
                            "date",
                            "popularity",
                            "alphabetical"
                        ],
                        "description": "How to sort results. Pass null if not needed."
                    }
                },
                "required": [
                    "num_results",
                    "domain_filter",
                    "sort_by"
                ],
                "additionalProperties": False
            }
        },
        "required": [
            "query",
            "options"
        ],
        "additionalProperties": False
    }
}]

response = client.responses.create(
    model="gpt-4.1",
    input=[{"role": "user", "content": "Can you find information about ChatGPT in the AI knowledge base?"}],
    tools=tools
)

print(response.output)
print(f"test: {response.output[0].arguments}")

[ResponseFunctionToolCall(arguments='{"query":"ChatGPT","options":{"num_results":5,"domain_filter":"AI","sort_by":"relevance"}}', call_id='call_oJ3ObPonrWOJcnsYFsySMdA8', name='search_knowledge_base', type='function_call', id='fc_682a23ebd0948191bd7bc9b3bf1637ae0dbb91dd3a8c57b0', status='completed')]
test: {"query":"ChatGPT","options":{"num_results":5,"domain_filter":"AI","sort_by":"relevance"}}
